In [9]:
import geopandas
import sys
# sys.path.append('../pystare/')
import pystare
import pandas

import geodata as gd

from pyhdf.SD import SD, SDC
from netCDF4 import Dataset


In [2]:
def divert_stderr():
    sys.stderr = open('stderr.out','w')  
    return

def restore_stderr():
    sys.stderr.close()
    with open('stderr.out') as f:
        count = sum(1 for _ in f)
    if count > 0:
        print(count, 'warnings or errors encountered while stderr diverted. See stderr.out.')
    sys.stderr = sys.__stderr__
    return

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import cartopy.crs as ccrs
import cartopy.feature as cf

import numpy
import shapely

# Some helper functions for plotting & printing.

class figax_container(object):
    def __init__(self,figax):
        self.fig = figax[0]
        self.ax  = figax[1]
        return

def add_coastlines(figax,set_global=False):
    "Add coastlines to the plot."
    ax = figax.ax
    if set_global:
        ax.set_global()
    ax.coastlines()
    return figax

def hello_plot(
        spatial_index_values=None
        ,figax=None
        ,plot_options={'projection':ccrs.PlateCarree(),'transform':ccrs.Geodetic()}
        ,set_global=False
        ,set_coastlines=True
        ,show=True
        ,color=None
        ,lw=1
        ):

    if figax is None:
        figax = figax_container(plt.subplots(1,subplot_kw=plot_options))
        if set_global:
            figax.ax.set_global()
        if set_coastlines:
            figax.ax.coastlines()
    else:
        ax = figax.ax
    
    if spatial_index_values is not None:
        # Calculate vertices and interconnection matrix
        lons,lats,intmat = pystare.triangulate_indices(spatial_index_values)
        
        # Make triangulation object & plot
        siv_triang = tri.Triangulation(lons,lats,intmat)
        # print('plot type triang: ',type(siv_triang))
        divert_stderr()
        figax.ax.triplot(siv_triang,c=color,transform=plot_options['transform'],lw=lw)
        restore_stderr()
    
    if show:
        plt.show()
        
    return figax

def hex16(i):
    return "0x%016x"%i

def lonlat_from_coords(coords):
    tmp = numpy.array(coords)
    lat=tmp[:,1]
    lon=tmp[:,0]
    return lon,lat

km  = 1 # Unit of length
deg = 1 # Unit of angle

# Set up the projection and transformation
proj         = ccrs.PlateCarree()
# proj        = ccrs.Robinson() # Drives matplotlib bug.
# proj        = ccrs.Mollweide() # Drives matplotlib bug.
transf       = ccrs.Geodetic()
plot_options = {'projection':proj,'transform':transf}

default_dpi = mpl.rcParamsDefault['figure.dpi']
mpl.rcParams['figure.dpi'] = 1.5*default_dpi

In [4]:
def load_modis():

    dataPath="extras/"
    
    # Retrieve the following files and place in the dataPath.
    # MOD03.A2005349.2125.061.2017187042720.hdf  MOD05_L2.A2005349.2125.061.2017294065400.hdf
    
    modis_base   = "MOD05_L2."
    
    modis_item       = "A2005349.2125.061.2017294065400"
    modis_time_start = "2005-12-15T21:25:00"
    modis_geofilename = "MOD03.A2005349.2125.061.2017187042720.hdf"
    
    modis_suffix = ".hdf"
    modis_filename = modis_base+modis_item+modis_suffix
    
    fmt_suffix = ".h5"
    workFileName = "sketchG."+modis_base+modis_item+fmt_suffix
    
    key_across = 'Cell_Across_Swath_1km:mod05'
    key_along  = 'Cell_Along_Swath_1km:mod05'

    print('loading: ',dataPath+modis_filename)
    hdf        = SD(dataPath+modis_filename,SDC.READ)
    ds_wv_nir  = hdf.select('Water_Vapor_Near_Infrared')
    data       = ds_wv_nir.get()
    
    
    # MODIS_Swath_Type_GEO/Geolocation_Fields/
    # Latitude
    
    hdf_geo           = SD(dataPath+modis_geofilename,SDC.READ)
    print('hg info: ',hdf_geo.info())
    # for idx,sds in enumerate(hdf_geo.datasets().keys()):
    #     print(idx,sds)
    # hdf_geo_ds = hdf_geo.select['']
    
    # hdf_geo_swath     = hdf_geo.select('MODIS_Swath_Type_GEO')
    # hdf_geo_swath_gf  = hdf_geo_swath['Geolocation_Fields']
    hdf_geo_lat       = hdf_geo.select('Latitude').get()
    hdf_geo_lon       = hdf_geo.select('Longitude').get()
    print('hgl type  ',type(hdf_geo_lat))
    print('hgl shape ',hdf_geo_lat.shape,hdf_geo_lon.shape)
    print('hgl dtype ',hdf_geo_lat.dtype)
    # exit()
    
    add_offset   = ds_wv_nir.attributes()['add_offset']
    scale_factor = ds_wv_nir.attributes()['scale_factor']
    print('scale_factor = %f, add_offset = %f.'%(scale_factor,add_offset))
    data = (data-add_offset)*scale_factor
    print('data mnmx: ',numpy.amin(data),numpy.amax(data))
    
    nAlong  = ds_wv_nir.dimensions()[key_along]
    nAcross = ds_wv_nir.dimensions()[key_across]
    print('ds_wv_nir nAlong,nAcross: ',nAlong,nAcross)
    
    dt = numpy.array([modis_time_start],dtype='datetime64[ms]')
    t_resolution = 26 # 5 minutes resolution? 2+6+10+6+6
    tid = pystare.from_utc(dt.astype(numpy.int64),t_resolution)
    # print(numpy.arange(numpy.datetime64("2005-12-15T21:20:00"),numpy.datetime64("2005-12-15T21:25:00")))
    # exit()
    
    fill_value = ds_wv_nir.attributes()['_FillValue']

    mod_lat = hdf_geo_lat.flatten()
    mod_lon = hdf_geo_lon.flatten()
    mod_dat = data.flatten()

    return mod_lon,mod_lat,mod_dat # load_modis


In [5]:
mod_lon,mod_lat,mod_dat = load_modis()

loading:  extras/MOD05_L2.A2005349.2125.061.2017294065400.hdf
hg info:  (46, 27)
hgl type   <class 'numpy.ndarray'>
hgl shape  (2030, 1354) (2030, 1354)
hgl dtype  float32
scale_factor = 0.001000, add_offset = 0.000000.
data mnmx:  -9.999000474927016 6.11300029035192
ds_wv_nir nAlong,nAcross:  2030 1354


In [6]:

def load_goes():

    # DEPENDENCIES
    goes_base    = "goes10."
    # goes_time    = "2005.349.003015"
    goes_time    = "2005.349.213015"

    ### GOES DATASET
    # Note GOES 10 (K) Bands 2-5 are 4-km, Band 1 (visible) is 1 km.
    goes_suffix = ".nc"
    goes_b5_dataPath = "extras/"
    goes_b5_dataFile = goes_base+goes_time+".BAND_05"+goes_suffix
    goes_b5_fqFilename = goes_b5_dataPath+goes_b5_dataFile
    goes_b5_ds = Dataset(goes_b5_fqFilename)

    g5shape = goes_b5_ds['data'].shape
    print('g5shape = ',g5shape)
    
    g5size = goes_b5_ds['data'].size
    print('g5size = ',g5size)
    
    # goes_b5_tid = gd.goes10_img_stare_time(goes_b5_ds)

    g5_dat = goes_b5_ds['data'][:,:].flatten()
    g5_lat = goes_b5_ds['lat'][:,:].flatten()
    g5_lon = goes_b5_ds['lon'][:,:].flatten()
    g5_idx_valid = numpy.where((g5_lat>=-90.0) & (g5_lat<=90.0))
    g5_idx_invalid = numpy.where(((g5_lat<-90.0) | (g5_lat>90.0)))
    g5_lon = g5_lon[g5_idx_valid]
    g5_lat = g5_lat[g5_idx_valid]
    g5_dat = g5_dat[g5_idx_valid]
    
    return g5_lon,g5_lat,g5_dat


In [7]:
g5_lon,g5_lat,g5_dat = load_goes()

g5shape =  (1, 1356, 3312)
g5size =  4491072
